In [51]:
## Importing packages
library(tidyverse) 
library(MASS)
library(tableone)
library(Matching)
library(MatchIt)
library(haven)
# The Path
#list.files(path = "../input")

 ## Loading RHC data set from Vanderbilt  and Summary of Data

In [52]:
load(url("http://biostat.mc.vanderbilt.edu/wiki/pub/Main/DataSets/rhc.sav"))

In [53]:
# Number of Paitentns=5735
# Number of variables=62

**Treatment Variable is Swang1 and Covariates are:
**
1. Cat1 which is Primary Disease 
2. Sex
3. Age
4. meanbp1: Mean Blood Pressure

 **Create a Data Set With Chosen Variables**

In [54]:
ARF<-as.numeric(rhc$cat1=='ARF')
CHF<-as.numeric(rhc$cat1=='CHF')
Cirr<-as.numeric(rhc$cat1=='Cirrhosis')
colcan<-as.numeric(rhc$cat1=='Colon Cancer')
Coma<-as.numeric(rhc$cat1=='Coma')
COPD<-as.numeric(rhc$cat1=='COPD')
lungcan<-as.numeric(rhc$cat1=='Lung Cancer')
MOSF<-as.numeric(rhc$cat1=='MOSF w/Malignancy')
sepsis<-as.numeric(rhc$cat1=='MOSF w/Sepsis')
female<-as.numeric(rhc$sex=='Female')
died<-as.numeric(rhc$death=='Yes')
age<-rhc$age
treatment<-as.numeric(rhc$swang1=='RHC')
meanbp1<-rhc$meanbp1
############ Here we make our data ##########
mydata<-cbind(ARF,CHF,Cirr,colcan,Coma,lungcan,MOSF,sepsis,age,female,meanbp1,treatment,died)
mydata<-data.frame(mydata)
############ Here are the covariates we use #################
xvars<-c("ARF","CHF","Cirr","colcan","Coma","lungcan","MOSF","sepsis",
         "age","female","meanbp1")

**Stratify Data Based on The Treatment**
1. Report the number of data points for Control & Treatment group
2. Report Mean & SD for each covariate
3. Report Standardized Mean Difference for Each Covariate

In [55]:
table1<- CreateTableOne(vars=xvars,strata="treatment", data=mydata, test=FALSE)
## include standardized mean difference (SMD)
print(table1,smd=TRUE)

                     Stratified by treatment
                      0             1             SMD   
  n                    3551          2184               
  ARF (mean (SD))      0.45 (0.50)   0.42 (0.49)   0.059
  CHF (mean (SD))      0.07 (0.25)   0.10 (0.29)   0.095
  Cirr (mean (SD))     0.05 (0.22)   0.02 (0.15)   0.145
  colcan (mean (SD))   0.00 (0.04)   0.00 (0.02)   0.038
  Coma (mean (SD))     0.10 (0.29)   0.04 (0.20)   0.207
  lungcan (mean (SD))  0.01 (0.10)   0.00 (0.05)   0.095
  MOSF (mean (SD))     0.07 (0.25)   0.07 (0.26)   0.018
  sepsis (mean (SD))   0.15 (0.36)   0.32 (0.47)   0.415
  age (mean (SD))     61.76 (17.29) 60.75 (15.63)  0.061
  female (mean (SD))   0.46 (0.50)   0.41 (0.49)   0.093
  meanbp1 (mean (SD)) 84.87 (38.87) 68.20 (34.24)  0.455


** Greedy Matching on Mahalanobis Distance & Outcome**


In [56]:
greedymatch<-Match(Tr=treatment,M=1,X=mydata[xvars],replace=FALSE)
matched<-mydata[unlist(greedymatch[c("index.treated","index.control")]), ]

In [67]:
matchedtab1<-CreateTableOne(vars=xvars,strata="treatment",data=matched,test=FALSE)
print(matchedtab1,smd=TRUE)

                     Stratified by treatment
                      0             1             SMD   
  n                    2184          2184               
  ARF (mean (SD))      0.42 (0.49)   0.42 (0.49)   0.006
  CHF (mean (SD))      0.10 (0.29)   0.10 (0.29)  <0.001
  Cirr (mean (SD))     0.02 (0.15)   0.02 (0.15)  <0.001
  colcan (mean (SD))   0.00 (0.02)   0.00 (0.02)  <0.001
  Coma (mean (SD))     0.04 (0.20)   0.04 (0.20)  <0.001
  lungcan (mean (SD))  0.00 (0.05)   0.00 (0.05)  <0.001
  MOSF (mean (SD))     0.07 (0.26)   0.07 (0.26)  <0.001
  sepsis (mean (SD))   0.24 (0.43)   0.32 (0.47)   0.177
  age (mean (SD))     61.53 (16.15) 60.75 (15.63)  0.049
  female (mean (SD))   0.44 (0.50)   0.41 (0.49)   0.042
  meanbp1 (mean (SD)) 73.12 (34.28) 68.20 (34.24)  0.144


## Run a Paired T-test & McNemare Test on the Outcome(Died)

In [112]:
y_trt<-matched$died[matched$treatment==1]
y_cont<-matched$died[matched$treatment==0]

In [114]:
# Paired T-test
t.test(y_trt-y_cont)


	One Sample t-test

data:  y_trt - y_cont
t = 3.0552, df = 1931, p-value = 0.00228
alternative hypothesis: true mean is not equal to 0
95 percent confidence interval:
 0.01686593 0.07733697
sample estimates:
 mean of x 
0.04710145 


In [94]:
# McNemar Test
mcnemar.test(table(y_trt,y_cont))


	McNemar's Chi-squared test with continuity correction

data:  table(y_trt, y_cont)
McNemar's chi-squared = 15.076, df = 1, p-value = 0.0001033


# Fit a Propensity Score Model (Using Logistic Regression)

In [98]:
# Here, we run the 
psmodel<-glm(treatment~ARF+CHF+Cirr+colcan+Coma+lungcan+MOSF+
               sepsis+age+female+meanbp1,
    family=binomial(),data=mydata)

In [118]:
## Comupte the propensity score & its logit
pscore<-psmodel$fitted.values
logit<-function(p){log(p)-log(1-p)}
##
psmatch<-Match(Tr=mydata$treatment,M=1,X=logit(pscore),replace=FALSE,caliper=0.2)
matched_propensity<-mydata[unlist(psmatch[c("index.treated","index.control")]),]

In [119]:
matchedtable_propensity<-CreateTableOne(vars=xvars,strata="treatment",data=matched_propensity,test=FALSE);
print(matchedtable_propensity,smd=TRUE)

                     Stratified by treatment
                      0             1             SMD   
  n                    1931          1931               
  ARF (mean (SD))      0.47 (0.50)   0.47 (0.50)   0.011
  CHF (mean (SD))      0.10 (0.30)   0.09 (0.29)   0.014
  Cirr (mean (SD))     0.03 (0.17)   0.03 (0.16)   0.028
  colcan (mean (SD))   0.00 (0.02)   0.00 (0.02)  <0.001
  Coma (mean (SD))     0.04 (0.20)   0.05 (0.22)   0.027
  lungcan (mean (SD))  0.00 (0.05)   0.00 (0.05)   0.011
  MOSF (mean (SD))     0.08 (0.27)   0.08 (0.27)   0.006
  sepsis (mean (SD))   0.25 (0.43)   0.25 (0.43)   0.008
  age (mean (SD))     61.04 (17.75) 60.93 (15.50)  0.007
  female (mean (SD))   0.44 (0.50)   0.43 (0.49)   0.020
  meanbp1 (mean (SD)) 71.40 (33.62) 70.98 (35.03)  0.012


In [120]:
y_trt_propensity=matched_propensity$died[matched_propensity$treatment==1];
y_con_propensity=matched_propensity$died[matched_propensity$treatment==0]

## Outcome Analysis--Paired T-test & Mcnemar Test

In [126]:
diffy_propensity<-y_trt_propensity-y_con_propensity;
t.test(diffy_propensity)
#table(y_trt_propensity,y_con_propensity)



	One Sample t-test

data:  diffy_propensity
t = 3.022, df = 1930, p-value = 0.002544
alternative hypothesis: true mean is not equal to 0
95 percent confidence interval:
 0.01617885 0.07600137
sample estimates:
 mean of x 
0.04609011 


In [129]:
table(y_trt_propensity,y_con_propensity)
mcnemar.test(table(y_trt_propensity,y_con_propensity))

                y_con_propensity
y_trt_propensity   0   1
               0 225 391
               1 480 835


	McNemar's Chi-squared test with continuity correction

data:  table(y_trt_propensity, y_con_propensity)
McNemar's chi-squared = 8.8909, df = 1, p-value = 0.002866
